# FID with GAN generation

In [1]:
from __future__ import print_function

import argparse
import math
import os
import random

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
from torchfusion.gan.applications import DCGANDiscriminator

from data_loader import MarioDataset
from models.custom import Generator

import csv

from image_gen.asset_map import get_asset_map
from image_gen.fixer import PipeFixer
from image_gen.image_gen import GameImageGenerator
from tqdm import tqdm

from get_level import GetLevel as getLevel
from scipy.linalg import sqrtm

### Functions for FID

In [2]:
# Function to preprocess the matrices
def preprocess_matrices(matrices):
    # Normalize values to the range [0, 255]
    normalized_matrices = (matrices - np.min(matrices)) / (np.max(matrices) - np.min(matrices))
    normalized_matrices = normalized_matrices * 255
    return normalized_matrices.astype(np.uint8)

# Function to compute mean and covariance of features
def compute_statistics(matrices):
    # Flatten matrices into vectors
    flattened_matrices = matrices.reshape((matrices.shape[0], -1))
    # Compute mean and covariance
    mean = np.mean(flattened_matrices, axis=0)
    covariance = np.cov(flattened_matrices, rowvar=False)

    return mean, covariance

# Function to compute Fréchet distance
def compute_frechet_distance(real_mean, real_cov, generated_mean, generated_cov):
    epsilon = 1e-6  # Small constant to avoid numerical instability
    sqrt_cov_product = sqrtm(real_cov.dot(generated_cov))
    fid_score = np.linalg.norm(real_mean - generated_mean) + np.trace(real_cov + generated_cov - 2 * sqrt_cov_product)

    return fid_score

## Batches for Real samples

In [3]:
org_data = MarioDataset()
ref_idx = torch.randperm(len(org_data))
prev_frame, curr_frame = (org_data[:].prev_frame, org_data[:].curr_frame)
complete_frame = torch.cat((prev_frame,curr_frame),dim=3)
complete_frame = torch.argmax(complete_frame, dim = 1)
#complete_frame = torch.tensor(complete_frame,dtype=torch.uint8)

[[[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 6]
  [2 2 2 ... 2 2 8]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 6 7]
  [2 2 2 ... 2 8 9]
  [0 0 0 ... 0 0 0]]

 ...

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 0]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 0 2]
  [0 0 0 ... 0 0 0]]]


In [4]:
# Preprocess real data
complete_frame_np = complete_frame.detach().numpy()
real_matrices = preprocess_matrices(complete_frame_np)
# Compute statistics for real matrices
real_mean, real_cov = compute_statistics(real_matrices)


### Batches for generated data

In [5]:

conditional_channels = conditional_channels = [0,1,6,7]
dataset = MarioDataset()
netG = Generator(
        latent_size=(len(conditional_channels) + 1, 14, 14), out_size=(13, 32, 32)
    )
netG.load_state_dict(torch.load("./trained_models/netG_epoch_300000_0_32.pth"))
    # 300000
mario_map = get_asset_map(game="mario")
gen = GameImageGenerator(asset_map=mario_map)
prev_frame, curr_frame = dataset[[120]]
fixer = PipeFixer()

level_gen = getLevel(netG, gen, fixer, prev_frame, curr_frame, conditional_channels)
var = 1
#noise = np.rand((1, 1, 14, 14)).normal_(0, var)
noise = np.random.normal(0,var,size=(14,14))
level = level_gen.generate_frames(noise, var=var, frame_count=1) # generated matrix without padded
padded = torch.zeros(32,32)
padded[9:-9,2:-2] = torch.from_numpy(level)
level = padded
level_finalize = torch.zeros(1,32,32)
level_finalize[0,:,:] = level
level = level_finalize

# this is just for visualization
level_gen.gen.save_gen_level(img_name="test_fuc_gen")

[[[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 6]
  [2 2 2 ... 2 2 8]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 6 7]
  [2 2 2 ... 2 8 9]
  [0 0 0 ... 0 0 0]]

 ...

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 0]
  [0 0 0 ... 0 0 0]]

 [[2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 2 2]
  ...
  [2 2 2 ... 2 2 2]
  [2 2 2 ... 2 0 2]
  [0 0 0 ... 0 0 0]]]


/home/ycv5080/miniconda3/envs/cs449/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [6]:
torch.cuda.is_available()

False

In [ ]:
# Preprocess generated data
generated_frame_np = level.detach().numpy()
generated_matrices = preprocess_matrices(generated_frame_np)
# Compute statistics for real matrices
gen_mean, gen_cov = compute_statistics(generated_matrices)

In [ ]:
# Compute Fréchet distance
fid_score = compute_frechet_distance(real_mean, real_cov, gen_mean, gen_cov)
print("FID score:", fid_score)

NameError: name 'real_mean' is not defined